In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

Cоздать модель с помощью RandomForest и xgboost, оптимизировать параметры.
Метрика - Root Mean Squared Logarithmic Error

In [3]:
df = pd.read_csv("train.csv")

In [4]:
df

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.14,0,0.00,3
1,3,1110,7,3301,15766,1216,4,33.52,0,0.00,4
2,3,1110,7,3301,15766,1238,4,39.32,0,0.00,4
3,3,1110,7,3301,15766,1240,4,33.52,0,0.00,4
4,3,1110,7,3301,15766,1242,3,22.92,0,0.00,3
...,...,...,...,...,...,...,...,...,...,...,...
74180459,9,25759,1,5517,4528866,32873,4,27.76,0,0.00,4
74180460,9,25759,1,5517,4528866,34226,4,37.04,0,0.00,4
74180461,9,25759,1,5517,4528866,45112,4,51.00,0,0.00,4
74180462,9,25759,1,5517,4547943,40217,3,26.94,3,26.94,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74180464 entries, 0 to 74180463
Data columns (total 11 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Semana             int64  
 1   Agencia_ID         int64  
 2   Canal_ID           int64  
 3   Ruta_SAK           int64  
 4   Cliente_ID         int64  
 5   Producto_ID        int64  
 6   Venta_uni_hoy      int64  
 7   Venta_hoy          float64
 8   Dev_uni_proxima    int64  
 9   Dev_proxima        float64
 10  Demanda_uni_equil  int64  
dtypes: float64(2), int64(9)
memory usage: 6.1 GB


In [6]:
short_df = df.head(10000)
short_df.to_csv('short.csv')

In [7]:
short_df

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.14,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.52,0,0.0,4
2,3,1110,7,3301,15766,1238,4,39.32,0,0.0,4
3,3,1110,7,3301,15766,1240,4,33.52,0,0.0,4
4,3,1110,7,3301,15766,1242,3,22.92,0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...
9995,3,1111,1,1004,49931,1125,9,86.40,0,0.0,9
9996,3,1111,1,1004,49931,1129,4,70.40,0,0.0,4
9997,3,1111,1,1004,49931,1146,6,128.34,0,0.0,6
9998,3,1111,1,1004,49931,1150,4,55.84,0,0.0,4


In [8]:
short_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Semana             10000 non-null  int64  
 1   Agencia_ID         10000 non-null  int64  
 2   Canal_ID           10000 non-null  int64  
 3   Ruta_SAK           10000 non-null  int64  
 4   Cliente_ID         10000 non-null  int64  
 5   Producto_ID        10000 non-null  int64  
 6   Venta_uni_hoy      10000 non-null  int64  
 7   Venta_hoy          10000 non-null  float64
 8   Dev_uni_proxima    10000 non-null  int64  
 9   Dev_proxima        10000 non-null  float64
 10  Demanda_uni_equil  10000 non-null  int64  
dtypes: float64(2), int64(9)
memory usage: 859.5 KB


In [14]:
print([len(set(short_df[i])) for i in short_df.columns])

[1, 2, 4, 34, 1607, 177, 171, 1656, 38, 126, 171]


In [15]:
len(set(short_df['Cliente_ID']))

1607

Semana — Week number (From Thursday to Wednesday)
Agencia_ID — Sales Depot ID
Canal_ID — Sales Channel ID
Ruta_SAK — Route ID (Several routes = Sales Depot)
Cliente_ID — Client ID
NombreCliente — Client name
Producto_ID — Product ID
NombreProducto — Product Name
Venta_uni_hoy — Sales unit this week (integer)
Venta_hoy — Sales this week (unit: pesos)
Dev_uni_proxima — Returns unit next week (integer)
Dev_proxima — Returns next week (unit: pesos)
Demanda_uni_equil — Adjusted Demand (integer) (This is the target you will predict)

In [25]:
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

In [21]:
main_df = short_df[['Venta_uni_hoy', 'Venta_hoy', 'Dev_uni_proxima', 'Dev_proxima', 'Demanda_uni_equil']]

In [44]:
def gridsearch_for_all_data(df, template_model, param_grid):
    model = template_model
    X = df.drop("Demanda_uni_equil", axis=1)
    y = df['Demanda_uni_equil']
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=0.8, random_state = 42)
    grid_model = GridSearchCV(model, param_grid)
    print('*************************************')
    grid_model.fit(X_train, Y_train)
    preds = grid_model.predict(X_test)
    print(grid_model.best_params_)
    print(grid_model.best_estimator_)
    print('RMSLE')
    print(np.sqrt(mean_squared_log_error(Y_test, [abs(i) for i in preds])))

In [36]:
from sklearn.ensemble import RandomForestRegressor

param_grid = {'n_estimators': (20, 50, 100), 'max_depth': (7, 10), 'min_samples_split': (2, 3)}


gridsearch_for_all_data(main_df, RandomForestRegressor(), param_grid)

*************************************
{'max_depth': 10, 'min_samples_split': 3, 'n_estimators': 50}
RandomForestRegressor(max_depth=10, min_samples_split=3, n_estimators=50)
RMSLE
0.03230521438395322


In [38]:
np.sqrt(mean_squared_log_error([0, 0, 0], [100, 200, 100]))

4.855365338735481

In [39]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     --------------------------------------- 89.1/89.1 MB 10.4 MB/s eta 0:00:00


In [40]:
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_california_housing
import xgboost as xgb
import multiprocessing

In [45]:
param_grid = {'eta': (0.3, 0.5, 0.1), 'max_depth': (7, 10)}


gridsearch_for_all_data(main_df, xgb.XGBRegressor(), param_grid)

*************************************
{'eta': 0.1, 'max_depth': 7}
XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False, eta=0.1,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.100000001, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=7, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', ...)
RMSLE
0.02698150630230331
